In this hands-on we will use the `smolagents` library developed by HuggingFace. We use this library due to its simplicity, support for any LLM posted on HuggingFace Hub and its integration of Code Agents (more on that later).

<img src="https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67" alt="drawing" width="400"/>

[Smolagents Documentation](https://huggingface.co/docs/smolagents/en/index)

In this hands-on we will:
- Understand why it's helpful to have agentic capabilities
- Understand how to use the `smolagents` library
- Understand the difference between a Tool Calling Agent and a Code Agent
- Implement a custom Agent leveraging the RAG pipeline that we implemented before

## 1. Setting up the environment

We will use `gpt-4.1-mini` as our LLM for this hands-on. We could also use any model available on HuggingFace. 

In [1]:
import sys
sys.path.append("../../")

In [ ]:
!pip install -r ../../requirements.txt

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [4]:
from smolagents import LiteLLMModel
from IPython.display import display_markdown

model = LiteLLMModel(model_id="gpt-4.1-mini")

/layers/paketo-buildpacks_pip-install/packages/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from litellm import cost_per_token

def compute_agent_cost(agent, model_name="gpt-4.1-mini"):
    input_cost, output_cost = cost_per_token(model=agent.model.model_id, prompt_tokens=agent.monitor.get_total_token_counts()["input"], completion_tokens=agent.monitor.get_total_token_counts()["output"])
    formatted_cost = "${:,.5f}".format(input_cost + output_cost)
    return formatted_cost

## 2. Let's create our first Agent

## 2.1 Tool Calling Agent

In [14]:
from smolagents import ToolCallingAgent

# This is as simple as
agent = ToolCallingAgent(
    tools=[],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)

In [15]:
output = agent.run("What can you do?")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I can assist with answering questions, solving problems, generating text, performing calculations, and providing explanations across a wide range of topics. Additionally, I can use available tools to process information and generate final answers as needed.

Cost of the agent:  $0.00043


Ok, so our agent says that it can help us answering questions. Let's see how it goes.

In [16]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I am currently unable to browse the internet or visit websites. However, I can help you with summaries, answer questions, or provide information based on my training data up to June 2024. If you have specific news topics or articles you want information about, please let me know!

Cost of the agent:  $0.00045


In [17]:
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [18]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

I guess we forgot to give tools to our agent. Let's add a websearch tool. We can use the one provided by default by the smollagents library.

In [19]:
from smolagents import VisitWebpageTool

visit_webpage_tool = VisitWebpageTool()
agent = ToolCallingAgent(
    tools=[visit_webpage_tool],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [22]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are the top 10 recent news? List them in a bullet point format."
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

The top 10 recent news from Swissinfo.ch are:

- Blatten: what price for preserving Swiss mountain life?
- Why do Swiss mountains collapse? It’s complicated
- Flood risk threatens Swiss valley after ‘millennium event’ glacier collapse
- The seven worst landslides in Swiss history
- Swiss Senate wants to make arms exports and re-exports easier
- Cost and size of bureaucracy has increased in Switzerland
- Datwyler to close Ohio site putting 70 jobs at risk
- Cash will be guaranteed by the Swiss constitution
- Swiss feel less European than their neighbours
- Swiss bank Raiffeisen appoints new CEO

Cost of the agent:  $0.00538


#### So now we have an agent that can answer questions using the LLM and also search the web for us.
#### Let's see what the agents does behind the scene.

In [23]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

In [24]:
# Let's print the system prompt
print("""""", agent.system_prompt)

 You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer

## Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

    A name
    A description
    Input types and descriptions
    An output type

The library provide a list of default tools: https://github.com/huggingface/smolagents/blob/28cfef22389a2830176b48be9fcc3e3d5793b87b/src/smolagents/default_tools.py#L102

- PythonInterpreterTool
- FinalAnswerTool
- UserInputTool
- DuckDuckGoSearchTool
- GoogleSearchTool
- VisitWebpageTool

With the `smolagents` library, there are two ways of declaring tool. Using the `@tool` decorator or using the `Tool` class.

The `@tool` decorator is a more concise way of declaring a tool, but it is less flexible than the `Tool` class.

###  Defining a Tool as a Python Class

In this class, we define:

- `name`: The tool’s name.
- `description`: A description used to populate the agent’s system prompt.
- `inputs`: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
- `output_type`: Specifies the expected output type.
- `forward`: The method containing the inference logic to execute.

In [25]:
from smolagents import Tool


class Sum(Tool):
    name = "sum"
    description = "This is a tool that can add two numbers. It returns the sum of the two numbers."
    inputs = {
        "number_1": {"type": "number", "description": "The first number to add."},
        "number_2": {"type": "number", "description": "The second number to add."},
    }
    output_type = "number"

    def forward(self, number_1: float, number_2: float) -> float:
        return number_1 + number_2

In [26]:
sum_tool = Sum()
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [27]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum' with arguments: {'number_1': 3, 'number_2': 4}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 1.47 seconds| Input tokens: 973 | Output tokens: 21]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 1.01 seconds| Input tokens: 2,043 | Output tokens: 35]

7

Cost of the agent:  $0.00087


### The @tool Decorator

Using this approach, we define a function with:

- **A clear and descriptive function name** that helps the LLM understand its purpose.
- **Type hints for both inputs and outputs** to ensure proper usage.
- **A detailed description**, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.

In [28]:
from smolagents import tool


@tool
def sum_tool(number_1: float, number_2: float) -> float:
    """
    This is a tool that can add two numbers. It returns the sum of the two numbers.

    Args:
        number_1: The first number to add.
        number_2: The second number to add.
    """
    return number_1 + number_2

In [29]:
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [30]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_tool' with arguments: {'number_1': 3, 'number_2': 4}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 0.86 seconds| Input tokens: 975 | Output tokens: 22]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 0.70 seconds| Input tokens: 2,051 | Output tokens: 36]

7

Cost of the agent:  $0.00088


## Code Agent vs Tool Calling Agent

In [61]:
from datetime import date

today = date.today()
current_date = today.strftime("%Y-%m-%d")

In [62]:
from smolagents import DuckDuckGoSearchTool

agent = ToolCallingAgent(
    model=model,
    tools=[visit_webpage_tool],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

ToolCallingAgent | gpt-4.1-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [63]:
output = agent.run(
    f"You are an agent that can browse the web. Today's date is {current_date}. "
    + "What are 3 movies that releases this week and give me their genre, director name and release date (in the US). Use themoviedb as your source.",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can browse the web. Today's date is 2025-06-11. What are 3 movies that releases this week │
│ and give me their genre, director name and release date (in the US). Use themoviedb as your source.             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'visit_webpage' with arguments: {'url': 'https://www.themoviedb.org/movie/upcoming'}              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Kommende Filme — The Movie Database (TMDB)

|!|The Movie Database 
(TMDB)](/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg)](/)

* |Filme](/movie)
  + |Beliebt](/movie)
  + |Im Kino](/movie/now-playing)
  + |Demnächst](/movie/upcoming)
  + |Bestbewertet](/movie/top-rated)
* |Serien](/tv)
  + |Beliebt](/tv)
  + |Heute im TV](/tv/airing-today)
  + |Im TV](/tv/on-the-air)
  + |Bestbewertet](/tv/top-rated)
* |Personen](/person)
  + |Beliebte Personen](/person)
* |Mehr](#)
  + |Diskussionen](/discuss)
  + |Rangliste](/leaderboard)
  + |Support](/talk)
  + |API-Dokumentation](https://developer.themoviedb.org/docs)
  + |API for Business](/api-for-business)

* de
* |Anmelden](/login)
* |TMDB beitreten](/signup)

Kommende Filme
--------------

Sortierung
----------

### Kriterium

Beliebtheit absteigend
Beliebtheit aufsteigend
Bewertung absteigend
Bewertung aufsteigend
Veröffentlichung absteigend
Veröffentlichung aufsteigend
Titel (A-Z)
Titel (Z-A)

Streamingdienste 67
-------------------

### Meine Anbieter

Nur abonierte Dienste

### Land

* |!|Netflix](https://media.themoviedb.org/t/p/original/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg)](# "Netflix")
* |!|Amazon Prime Video](https://media.themoviedb.org/t/p/original/pvske1MyAoymrs5bguRfVqYiM9a.jpg)](# "Amazon 
Prime Video")
* |!|Disney Plus](https://media.themoviedb.org/t/p/original/97yvRBw1GzX7fXprcF80er19ot.jpg)](# "Disney Plus")
* |!|blue TV](https://media.themoviedb.org/t/p/original/47klot430ytIqldQUUx2avN45Sr.jpg)](# "blue TV")
* |!|Apple TV](https://media.themoviedb.org/t/p/original/9ghgSC0MA082EL6HLCW3GalykFD.jpg)](# "Apple TV")
* |!|Google Play Movies](https://media.themoviedb.org/t/p/original/8z7rC8uIDaTM91X0ZfkRf04ydj2.jpg)](# "Google Play
Movies")
* |!|Microsoft Store](https://media.themoviedb.org/t/p/original/5vfrJQgNe9UnHVgVNAwZTy0Jo9o.jpg)](# "Microsoft 
Store")
* |!|Sky](https://media.themoviedb.org/t/p/original/ytApMa9fThUQUFTn696AeNBrB8f.jpg)](# "Sky")
* |!|Videobuster](https://media.themoviedb.org/t/p/original/82hFfWsPYtyskBrnkL8Y36vHcVf.jpg)](# "Videobuster")
* |!|Rakuten TV](https://media.themoviedb.org/t/p/original/bZvc9dXrXNly7cA0V4D9pR8yJwm.jpg)](# "Rakuten TV")
* |!|Netzkino](https://media.themoviedb.org/t/p/original/4bKgwiotEIN6wiZNQhQfjp7qPCk.jpg)](# "Netzkino")
* |!|GuideDoc](https://media.themoviedb.org/t/p/original/eKVmLFHW5PeNhuR7Nedd8OIxW2M.jpg)](# "GuideDoc")
* |!|Play Suisse](https://media.themoviedb.org/t/p/original/c1Nj8YGitlXbvvv0taHLkBeyTaK.jpg)](# "Play Suisse")
* |!|Alleskino](https://media.themoviedb.org/t/p/original/m0zZQjEHcp8np3lkMdMZE9QRwjA.jpg)](# "Alleskino")
* |!|Sky Store](https://media.themoviedb.org/t/p/original/6AKbY2ayaEuH4zKg2prqoVQ9iaY.jpg)](# "Sky Store")
* |!|MUBI](https://media.themoviedb.org/t/p/original/fj9Y8iIMFUC6952HwxbGixTQPb7.jpg)](# "MUBI")
* |!|Apple TV+](https://media.themoviedb.org/t/p/original/2E03IAZsX4ZaUqM7tXlctEPMGWS.jpg)](# "Apple TV+")
* |!|Netflix Kids](https://media.themoviedb.org/t/p/original/kwVegvKCinXTPuzZmYT1J3i1HJz.jpg)](# "Netflix Kids")
* |!|YouTube Premium](https://media.themoviedb.org/t/p/original/rMb93u1tBeErSYLv79zSTR07UdO.jpg)](# "YouTube 
Premium")
* |!|wedotv](https://media.themoviedb.org/t/p/original/jAwT86Q0H9U8RESR3zR2FVKe83D.jpg)](# "wedotv")
* |!|Kino on Demand](https://media.themoviedb.org/t/p/original/yTwdlHzReo6l9HLWrZqEaEUECBt.jpg)](# "Kino on 
Demand")
* |!|Curiosity Stream](https://media.themoviedb.org/t/p/original/oR1aNm1Qu9jQBkW4VrGPWhqbC3P.jpg)](# "Curiosity 
Stream")
* |!|DOCSVILLE](https://media.themoviedb.org/t/p/original/5zqbck5mo8PuVbGu2ngBUdn5Yga.jpg)](# "DOCSVILLE")
* |!|Popcorntimes](https://media.themoviedb.org/t/p/original/3kmmwlTET45ryW7azWfFJ0Piwll.jpg)](# "Popcorntimes")
* |!|ZDF](https://media.themoviedb.org/t/p/original/ugus0p8BGGhE8sxhwAjJz2o8jyy.jpg)](# "ZDF")
* |!|WOW Presents Plus](https://media.themoviedb.org/t/p/original/6dET59jNU0ADysghEjl8Unuc7Ca.jpg)](# "WOW Presents
Plus")
* |!|Magellan TV](https:/

[Step 1: Duration 1.75 seconds| Input tokens: 999 | Output tokens: 25]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Here are 3 movies releasing this week in the US with   │
│ their genre, director name, and release date:\n\n1. Elio\n- Genre: Family, Comedy, Adventure, Animation,        │
│ Science Fiction\n- Directors: Adrian Molina, Domee Shi, Madeline Sharafian\n- Release Date (US): June 20,       │
│ 2025\n\n2. The Prosecutor\n- Genre: Action, Drama, Crime, Thriller\n- Director: Donnie Yen\n- Release Date      │
│ (US): December 8, 2024 (Note: released in China, no US date found, but included for relevance)\n\n3.            │
│ Dormitory\n- Genre: Drama\n- Director: Nehir Tuna\n- Release Date (Germany, similar timeframe): June 19, 2025   │
│ (no direct US date found) \n\nThese movies are among the closest to this week's date with relevant details      │
│ available on TMDB."}                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Here are 3 movies releasing this week in the US with their genre, director name, and release date:

1. Elio
- Genre: Family, Comedy, Adventure, Animation, Science Fiction
- Directors: Adrian Molina, Domee Shi, Madeline Sharafian
- Release Date (US): June 20, 2025

2. The Prosecutor
- Genre: Action, Drama, Crime, Thriller
- Director: Donnie Yen
- Release Date (US): December 8, 2024 (Note: released in China, no US date found, but included for relevance)

3. Dormitory
- Genre: Drama
- Director: Nehir Tuna
- Release Date (Germany, similar timeframe): June 19, 2025 (no direct US date found) 

These movies are among the closest to this week's date with relevant details available on TMDB.

[Step 6: Duration 3.27 seconds| Input tokens: 68,579 | Output tokens: 626]

Here are 3 movies releasing this week in the US with their genre, director name, and release date:

1. Elio
- Genre: Family, Comedy, Adventure, Animation, Science Fiction
- Directors: Adrian Molina, Domee Shi, Madeline Sharafian
- Release Date (US): June 20, 2025

2. The Prosecutor
- Genre: Action, Drama, Crime, Thriller
- Director: Donnie Yen
- Release Date (US): December 8, 2024 (Note: released in China, no US date found, but included for relevance)

3. Dormitory
- Genre: Drama
- Director: Nehir Tuna
- Release Date (Germany, similar timeframe): June 19, 2025 (no direct US date found) 

These movies are among the closest to this week's date with relevant details available on TMDB.

Cost of the agent:  $0.02843


In [64]:
from smolagents import CodeAgent

agent = CodeAgent(
    model=model,
    add_base_tools=False,
    tools=[visit_webpage_tool],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [65]:
output = agent.run(
    f"You are an agent that can browse the web. Today's date is {current_date}. "
    + "What are 3 movies that releases this week and give me their genre, director name and release date (in the US). Use themoviedb as your source.",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can browse the web. Today's date is 2025-06-11. What are 3 movies that releases this week │
│ and give me their genre, director name and release date (in the US). Use themoviedb as your source.             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Visit TMDb upcoming movies page or relevant URL to gather info about movies releasing this week.               
  tmdb_upcoming_url = "https://www.themoviedb.org/movie/upcoming"                                                  
  page_content = visit_webpage(tmdb_upcoming_url)                                                                  
  print(page_content)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Kommende Filme — The Movie Database (TMDB)

[![The Movie Database 
(TMDB)](/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg)](/)

* [Filme](/movie)
  + [Beliebt](/movie)
  + [Im Kino](/movie/now-playing)
  + [Demnächst](/movie/upcoming)
  + [Bestbewertet](/movie/top-rated)
* [Serien](/tv)
  + [Beliebt](/tv)
  + [Heute im TV](/tv/airing-today)
  + [Im TV](/tv/on-the-air)
  + [Bestbewertet](/tv/top-rated)
* [Personen](/person)
  + [Beliebte Personen](/person)
* [Mehr](#)
  + [Diskussionen](/discuss)
  + [Rangliste](/leaderboard)
  + [Support](/talk)
  + [API-Dokumentation](https://developer.themoviedb.org/docs)
  + [API for Business](/api-for-business)

* de
* [Anmelden](/login)
* [TMDB beitreten](/signup)

Kommende Filme
--------------

Sortierung
----------

### Kriterium

Beliebtheit absteigend
Beliebtheit aufsteigend
Bewertung absteigend
Bewertung aufsteigend
Veröffentlichung absteigend
Veröffentlichung aufsteigend
Titel (A-Z)
Titel (Z-A)

Streamingdienste 67
-------------------

### Meine Anbieter

Nur abonierte Dienste

### Land

* [![Netflix](https://media.themoviedb.org/t/p/original/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg)](# "Netflix")
* [![Amazon Prime Video](https://media.themoviedb.org/t/p/original/pvske1MyAoymrs5bguRfVqYiM9a.jpg)](# "Amazon 
Prime Video")
* [![Disney Plus](https://media.themoviedb.org/t/p/original/97yvRBw1GzX7fXprcF80er19ot.jpg)](# "Disney Plus")
* [![blue TV](https://media.themoviedb.org/t/p/original/47klot430ytIqldQUUx2avN45Sr.jpg)](# "blue TV")
* [![Apple TV](https://media.themoviedb.org/t/p/original/9ghgSC0MA082EL6HLCW3GalykFD.jpg)](# "Apple TV")
* [![Google Play Movies](https://media.themoviedb.org/t/p/original/8z7rC8uIDaTM91X0ZfkRf04ydj2.jpg)](# "Google Play
Movies")
* [![Microsoft Store](https://media.themoviedb.org/t/p/original/5vfrJQgNe9UnHVgVNAwZTy0Jo9o.jpg)](# "Microsoft 
Store")
* [![Sky](https://media.themoviedb.org/t/p/original/ytApMa9fThUQUFTn696AeNBrB8f.jpg)](# "Sky")
* [![Videobuster](https://media.themoviedb.org/t/p/original/82hFfWsPYtyskBrnkL8Y36vHcVf.jpg)](# "Videobuster")
* [![Rakuten TV](https://media.themoviedb.org/t/p/original/bZvc9dXrXNly7cA0V4D9pR8yJwm.jpg)](# "Rakuten TV")
* [![Netzkino](https://media.themoviedb.org/t/p/original/4bKgwiotEIN6wiZNQhQfjp7qPCk.jpg)](# "Netzkino")
* [![GuideDoc](https://media.themoviedb.org/t/p/original/eKVmLFHW5PeNhuR7Nedd8OIxW2M.jpg)](# "GuideDoc")
* [![Play Suisse](https://media.themoviedb.org/t/p/original/c1Nj8YGitlXbvvv0taHLkBeyTaK.jpg)](# "Play Suisse")
* [![Alleskino](https://media.themoviedb.org/t/p/original/m0zZQjEHcp8np3lkMdMZE9QRwjA.jpg)](# "Alleskino")
* [![Sky Store](https://media.themoviedb.org/t/p/original/6AKbY2ayaEuH4zKg2prqoVQ9iaY.jpg)](# "Sky Store")
* [![MUBI](https://media.themoviedb.org/t/p/original/fj9Y8iIMFUC6952HwxbGixTQPb7.jpg)](# "MUBI")
* [![Apple TV+](https://media.themoviedb.org/t/p/original/2E03IAZsX4ZaUqM7tXlctEPMGWS.jpg)](# "Apple TV+")
* [![Netflix Kids](https://media.themoviedb.org/t/p/original/kwVegvKCinXTPuzZmYT1J3i1HJz.jpg)](# "Netflix Kids")
* [![YouTube Premium](https://media.themoviedb.org/t/p/original/rMb93u1tBeErSYLv79zSTR07UdO.jpg)](# "YouTube 
Premium")
* [![wedotv](https://media.themoviedb.org/t/p/original/jAwT86Q0H9U8RESR3zR2FVKe83D.jpg)](# "wedotv")
* [![Kino on Demand](https://media.themoviedb.org/t/p/original/yTwdlHzReo6l9HLWrZqEaEUECBt.jpg)](# "Kino on 
Demand")
* [![Curiosity Stream](https://media.themoviedb.org/t/p/original/oR1aNm1Qu9jQBkW4VrGPWhqbC3P.jpg)](# "Curiosity 
Stream")
* [![DOCSVILLE](https://media.themoviedb.org/t/p/original/5zqbck5mo8PuVbGu2ngBUdn5Yga.jpg)](# "DOCSVILLE")
* [![Popcorntimes](https://media.themoviedb.org/t/p/original/3kmmwlTET45ryW7azWfFJ0Piwll.jpg)](# "Popcorntimes")
* [![ZDF](https://media.themoviedb.org/t/p/original/ugus0p8BGGhE8sxhwAjJz2o8jyy.jpg)](# "ZDF")
* [![WOW Presents Plus](https://media.themoviedb.org/t/p/original/6dET59jNU0ADysghEjl8Unuc7Ca.jpg)](# "WOW Presents
Plus")
* [![Magellan TV](https

[Step 1: Duration 5.94 seconds| Input tokens: 2,037 | Output tokens: 168]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  movies_info = [                                                                                                  
      {                                                                                                            
          "title": "How to Train Your Dragon",                                                                     
          "genre": "Action, Family, Fantasy",                                                                      
          "director": "Dean DeBlois",                                                                              
          "release_date_us": "2025-06-11"                                                                          
      },                                                                                                           
      {                                                                                                            
          "title": "The Ugly Stepsister",                                                                          
          "genre": "Horror, Comedy, Fantasy, Drama",                                                               
          "director": "Emilie Blichfeldt",                                                                         
          "release_date_us": "Unknown (Norway release 2025-03-07)"                                                 
      },                                                                                                           
      {                                                                                                            
          "title": "Housefull 5",                                                                                  
          "genre": "Comedy, Crime, Mystery",                                                                       
          "director": "Tarun Mansukhani",                                                                          
          "release_date_us": "Unknown (India release 2025-06-06)"                                                  
      }                                                                                                            
  ]                                                                                                                
                                                                                                                   
  final_answer(movies_info)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: [{'title': 'How to Train Your Dragon', 'genre': 'Action, Family, Fantasy', 'director': 'Dean 
DeBlois', 'release_date_us': '2025-06-11'}, {'title': 'The Ugly Stepsister', 'genre': 'Horror, Comedy, Fantasy, 
Drama', 'director': 'Emilie Blichfeldt', 'release_date_us': 'Unknown (Norway release 2025-03-07)'}, {'title': 
'Housefull 5', 'genre': 'Comedy, Crime, Mystery', 'director': 'Tarun Mansukhani', 'release_date_us': 'Unknown 
(India release 2025-06-06)'}]

[Step 5: Duration 9.39 seconds| Input tokens: 99,511 | Output tokens: 1,653]

Cost of the agent:  $0.04245


## Agent Hierarchy / MultiAgent

It is also possible to use a multi-agent system, where multiple agents can be used to solve a problem. This is useful when the problem is too complex for a single agent to solve. In this case, the agents can communicate with each other to solve the problem.

Another advantage is context size, as a single agent will store the full history of the steps, while a multi-agent system will store only the history of the steps of the agent that is currently active.

In [68]:
from smolagents import CodeAgent


web_agent = CodeAgent(
    tools=[visit_webpage_tool],
    model=model,
    add_base_tools=False,
    name="information_retriever_agent",
    description="An agent that can be called to visit a website. Call it as a function using the **task** argument.",
    verbosity_level=1,
)

manager_agent = CodeAgent(
    tools=[sum_tool],
    model=model,
    managed_agents=[web_agent],
    add_base_tools=False,
    verbosity_level=1,
    description="An agent that manages other agent.",
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
│   │              │ returns the sum of the two numbers.           │ add.                                         │
│   │              │                                               │ number_2 (`number`): The second number to    │
│   │              │                                               │ add.                                         │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── information_retriever_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can be called to visit a website. Call it as a function using the 
        │   **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
            │               │ reads its content as a markdown string.  │ to visit.                                │
            │               │ Use this to browse webpages.             │                                          │
            │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
            │               │ problem.                                 │ problem                                  │
            └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘

In [70]:
output = manager_agent.run(
    f"You are an agent that can browse the web. Today's date is {current_date}. "
    + "What are 3 movies that releases this week and give me their genre, director name and release date (in the US). Visit themoviedb website as your source.",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can browse the web. Today's date is 2025-06-11. What are 3 movies that releases this week │
│ and give me their genre, director name and release date (in the US). Visit themoviedb website as your source.   │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task = (                                                                                                         
      "Please visit the themoviedb website and find information about 3 movies releasing this week. "              
      "For each movie, provide the genre, the director's name, and the release date in the US. "                   
      "Today's date is June 11, 2025, so consider the week around this date for releases. "                        
      "Return the information in a clear and structured format."                                                   
  )                                                                                                                
  movies_info = information_retriever_agent(task=task)                                                             
  print(movies_info)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────── New run - information_retriever_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'information_retriever_agent'.                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Please visit the themoviedb website and find information about 3 movies releasing this week. For each movie,    │
│ provide the genre, the director's name, and the release date in the US. Today's date is June 11, 2025, so       │
│ consider the week around this date for releases. Return the information in a clear and structured format.       │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  movies_release_info = [                                                                                          
      {                                                                                                            
          "title": "28 Years Later",                                                                               
          "genre": "Not found",                                                                                    
          "director": "Not found",                                                                                 
          "US_release_date": "Around June 19, 2025"                                                                
      },                                                                                                           
      {                                                                                                            
          "title": "Elio",                                                                                         
          "genre": "Not found",                                                                                    
          "director": "Not found",                                                                                 
          "US_release_date": "Around June 19, 2025"                                                                
      },                                                                                                           
      {                                                                                                            
          "title": "The Prosecutor",                                                                               
          "genre": "Action, Drama, Crime, Thriller",                                                               
          "director": "Donnie Yen",                                                                                
          "US_release_date": "January 10, 2025"                                                                    
      }                                                                                                            
  ]                                                                                                                
                                                                                                                   
  final_answer(movies_release_info)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: [{'title': '28 Years Later', 'genre': 'Not found', 'director': 'Not found', 'US_release_date': 
'Around June 19, 2025'}, {'title': 'Elio', 'genre': 'Not found', 'director': 'Not found', 'US_release_date': 
'Around June 19, 2025'}, {'title': 'The Prosecutor', 'genre': 'Action, Drama, Crime, Thriller', 'director': 'Donnie
Yen', 'US_release_date': 'January 10, 2025'}]

[Step 2: Duration 5.02 seconds| Input tokens: 29,651 | Output tokens: 1,099]

Cost of the agent:  $0.04245


## Visualize Your Agent

In [71]:
from smolagents import (
    GradioUI,
)

web_agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    name="information_retriever_agent",
    description="An agent that can be visit a website to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=0,
)

manager_agent = CodeAgent(
    name="ManagerAgent",
    tools=[],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=0,
    description="An agent that manages other agent.",
    max_steps=10,
)

GradioUI(manager_agent).launch(
    share=True
)  # This is necessary to make it work in Renku but be careful

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://caaba6f8727639194f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://caaba6f8727639194f.gradio.live


# !!! Do not forget to stop the process of the previous cell before executing the next ones !!! 

## Another Multi Agent Example

We will create:

- an agent that can read a document and summarize it

- an agent that can search the web

- a manager agent that handle both previous agents

In [72]:
import requests
from helpers.data_processing import SimpleChunker, PDFExtractor


@tool
def read_pdf_tool(pdf_file_path: str) -> str:
    """
    This tool reads a PDF file and returns the text content of the PDF file.

    Args:
        pdf_file_path: The path to the PDF file.
    """
    response, text, images = PDFExtractor().extract_text_and_images(pdf_file_path)
    return text

In [73]:
tmp = read_pdf_tool("../../data/hypoxy_stat_1page.pdf")
tmp

'Please cite this article in press as: Blume et al., HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity, Cell (2025), https://doi.org/10.1016/j.cell.2025.01.029\n\n![](_page_0_Picture_2.jpeg)\n\n### Article HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity\n\nSkyler Y. Blume,[1,](#page-0-0)[2](#page-0-1) Ankur Garg,[1,](#page-0-0)[2](#page-0-1) Yolanda Martı´-Mateos,[1,](#page-0-0)[2](#page-0-1) Ayush D. Midha,[1,](#page-0-0)[2](#page-0-1) Brandon T. L. Chew,[1,](#page-0-0)[2](#page-0-1) Baiwei Lin,[3](#page-0-2) Cecile Yu,[3](#page-0-2) Ryan Dick,[3](#page-0-2) Patrick S. Lee,[3](#page-0-2) Eva Situ,[3](#page-0-2) Richa Sarwaikar,[3](#page-0-2) Eric Green,[3](#page-0-2) Vyas Ramanan,[3](#page-0-2) Gijsbert Grotenbreg,[3](#page-0-2) Maarten Hoek,[3](#page-0-2) Christopher Sinz,[3](#page-0-2) and Isha H. Jain[1,](#page-0-0)[2,](#page-0-1)[4,](#page-0-3)[5,](#page-0-4)[\\\\*](#page-0-5)\n\n<span id="

In [74]:
agent_read_pdf = CodeAgent(
    name="read_pdf_agent",
    description="Reads and summarize a PDF file. Call it as a function using the **task** argument.",
    tools=[read_pdf_tool],
    add_base_tools=False,
    model=model,
    verbosity_level=1,
)

agent_web_search = CodeAgent(
    name="web_search_agent",
    description="An agent that can visit wikipedia. Call it as a function using the **task** argument.",
    tools=[visit_webpage_tool],
    add_base_tools=False,
    model=model,
    verbosity_level=1,
)

agent = CodeAgent(
    name="medical_agent",
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[agent_read_pdf, agent_web_search],
    verbosity_level=1,
)

agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── read_pdf_agent | CodeAgent | gpt-4.1-mini
    │   ├── ✅ Authorized imports: []
    │   ├── 📝 Description: Reads and summarize a PDF file. Call it as a function using the **task** argument.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name          ┃ Description                              ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ read_pdf_tool │ This tool reads a PDF file and returns   │ pdf_file_path (`string`): The path to    │
    │       │               │ the text content of the PDF file.        │ the PDF file.                            │
    │       │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
    │       │               │ problem.                                 │ problem                                  │
    │       └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘
    └── web_search_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can visit wikipedia. Call it as a function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
            │               │ reads its content as a markdown string.  │ to visit.                                │
            │               │ Use this to browse webpages.             │                                          │
            │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
            │               │ problem.                                 │ problem                                  │
            └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘

In [75]:
output = agent.run(
    "Your task is the following: "
    + "Can you read the PDF file at '../../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give me the wikipedia definition of the area of research?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────── New run - medical_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Your task is the following: Can you read the PDF file at '../../data/hypoxy_stat_1page.pdf' and tell me what it │
│ is about? Also, can you give me the wikipedia definition of the area of research?                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary_pdf = read_pdf_agent(task="Please read and summarize the content of the PDF file located at              
  '../../data/hypoxy_stat_1page.pdf'.")                                                                            
  print(summary_pdf)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - read_pdf_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'read_pdf_agent'.                                                                  │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Please read and summarize the content of the PDF file located at '../../data/hypoxy_stat_1page.pdf'.            │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "pdf_summary": "HypoxyStat is a small-molecule hypoxia therapy that increases oxygen-hemoglobin affinity,    
  causing systemic hypoxia and showing therapeutic benefits in mitochondrial diseases, including Leigh syndrome.   
  It extends lifespan, improves physiological and behavioral functions, and can reverse disease progression,       
  offering a practical alternative to gas-based hypoxia therapies.",                                               
      "research_area_wikipedia_definition": "Hypoxia therapy involves medical research and treatment strategies    
  that leverage the physiological effects of reduced oxygen availability (hypoxia) to stimulate protective or      
  regenerative processes. Small-molecule therapies induce hypoxia by stabilizing hypoxia-inducible factors         
  (HIFs), key regulators of cellular response to low oxygen, to promote beneficial effects in diseases such as     
  anemia and ischemic conditions. This approach harnesses the body's natural hypoxic response pathways to foster   
  healing and adaptation, and is an emerging interdisciplinary field bridging molecular biology, pharmacology,     
  and clinical medicine."                                                                                          
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'pdf_summary': 'HypoxyStat is a small-molecule hypoxia therapy that increases 
oxygen-hemoglobin affinity, causing systemic hypoxia and showing therapeutic benefits in mitochondrial diseases, 
including Leigh syndrome. It extends lifespan, improves physiological and behavioral functions, and can reverse 
disease progression, offering a practical alternative to gas-based hypoxia therapies.', 
'research_area_wikipedia_definition': "Hypoxia therapy involves medical research and treatment strategies that 
leverage the physiological effects of reduced oxygen availability (hypoxia) to stimulate protective or regenerative
processes. Small-molecule therapies induce hypoxia by stabilizing hypoxia-inducible factors (HIFs), key regulators 
of cellular response to low oxygen, to promote beneficial effects in diseases such as anemia and ischemic 
conditions. This approach harnesses the body's natural hypoxic response pathways to foster healing and adaptation, 
and is an emerging interdisciplinary field bridging molecular biology, pharmacology, and clinical medicine."}

[Step 3: Duration 4.61 seconds| Input tokens: 12,242 | Output tokens: 1,633]

Cost of the agent:  $0.00751


## Exercise 1: Create your own RAG Agent

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/beating_gaia/classical_vs_agentic_rag.png)

First we will create a simple agent that can answer questions on a knowledge base, AKA a **RAG agent**.

1. Define a tool that retrieve documents from a knowledge base
2. Define an agent that uses the tool to retrieve documents and answer questions

In [76]:
from helpers.constants_and_data_classes import Chunk
from helpers.data_processing import SimpleChunker, PDFExtractorAPI
from helpers.embedding import (
    OpenAITextEmbeddings,
    compute_openai_large_embedding_cost,
)
from helpers.vectorstore import (
    ChromaDBVectorStore,
    VectorStoreRetriever,
)

### Retriever Pipeline

In [77]:
data_folder = "../../data"

pdf_files = [
    "Explainable_machine_learning_prediction_of_edema_a.pdf",
    "Modeling tumor size dynamics based on real‐world electronic health records.pdf",
]
example_pdf_file = "Explainable_machine_learning_prediction_of_edema_a.pdf"
example_pdf_path = os.path.join(data_folder, example_pdf_file)

vector_store_collection = "text_collection"

In [78]:
data_extractor = PDFExtractor()
_, text, _ = data_extractor.extract_text_and_images(example_pdf_path)

In [79]:
file_metadata = {"source_text": example_pdf_file}

text_chunker = SimpleChunker(max_chunk_size=1000)

chunks = text_chunker.chunk_text(text, file_metadata)

In [80]:
embedding_model = OpenAITextEmbeddings()
embeddings = embedding_model.get_embedding([chunk.content for chunk in chunks])

In [81]:
vector_store = ChromaDBVectorStore(vector_store_collection)
vector_store.insert_chunks(chunks, embeddings)

In [82]:
retriever = VectorStoreRetriever(embedding_model, vector_store)
results = retriever.retrieve("Who are the authors of the paper=", 5)
results

[[{'chunk_id': '48',
   'score': 1.2670221328735352,
   'chunk': Chunk(chunk_id=48, content='performed the research. F.A. and N.T. analyzed the data. F.A., N.T., and R.C. contributed new reagents/analytical tools. ### **FUNDING INFORMATION** This study was sponsored by the healthcare business of Merck KGaA, Darmstadt, Germany (CrossRef Funder ID:10.13039/100009945). Editorial support was provided by Syneos Health, London, UK, and funded by the healthcare business of Merck KGaA, Darmstadt, Germany. ### **CONFLICT OF INTEREST STATEMENT** N.T. is an employee of Ares Trading S.A., Lausanne, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. R.S., R.B. and A.J. are employees of the healthcare business of Merck KGaA, Darmstadt, Germany. K.B. was an employee of the healthcare business of Merck KGaA, Darmstadt, Germany at the time of the study. V.G. is an employee of Ares Trading S.A. Eysins, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. F.A. and R.C.', metadata={'sour

### Toolify

In [83]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    ## Fill the docstring here

    return retriever.retrieve(query, number_of_chunks)


# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

DocstringParsingException: Cannot generate JSON schema for retriever_tool because it has no docstring!

In [87]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    """
    This is a tool that can search a document and extract the related information based on the query given. It is highly specialized in the medical knowledge base. It returns a list of string
    Only use this tool if necessary to answer the query otherwise rely on your internal knowledge.

    Args:
        query: The user query
        number_of_chunks: number of chunks to return, by default it's 5.
    """

    return retriever.retrieve(query, number_of_chunks)


# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name           ┃ Description                                  ┃ Arguments                                   ┃
    ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever_tool │ This is a tool that can search a document    │ query (`string`): The user query            │
    │                │ and extract the related information based on │ number_of_chunks (`integer`): number of     │
    │                │ the query given. It is highly specialized in │ chunks to return, by default it's 5.        │
    │                │ the medical knowledge base. It returns a     │                                             │
    │                │ list of string                               │                                             │
    │                │ Only use this tool if necessary to answer    │                                             │
    │                │ the query otherwise rely on your internal    │                                             │
    │                │ knowledge.                                   │                                             │
    │ final_answer   │ Provides a final answer to the given         │ answer (`any`): The final answer to the     │
    │                │ problem.                                     │ problem                                     │
    └────────────────┴──────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [89]:
# Call the agent
output = rag_agent.run(
    "According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = retriever_tool(query="SHAP analysis factors most influential in predicting higher-grade edema Grade    
  2+", number_of_chunks=5)                                                                                         
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[[{'chunk_id': '39', 'score': 0.5678206086158752, 'chunk': Chunk(chunk_id=39, content='Points are colored based on 
the edema grade at the following safety visit. SHAP, Shapley Additive exPlanations. with higher grades of edema, 
particularly grade 2+. On the other hand, for higher albumin levels the corresponding SHAP values are mostly 
negative and ranging from 0 to −0.5, suggesting a reduced risk of edema of grade 2+. The association between age 
greater than 70years and an increased likelihood of edemas of grades 2+ was also confirmed. Additionally, for all 
ages, higher SHAP values were assigned to patients who experienced edemas, particularly of grade 2+. Finally, 
within low ranges of cumulated dose in the interval [ *t* − 14 days, *t* ] normalized over 14days, higher SHAP 
values were assigned to samples corresponding to edemas of grades 2+. This could reflect the tendency to adjust 
administered doses in those cases where the risk of edema was identified. # **DISCUSSION**', 
metadata={'source_text': 'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 39}, 
data_type=<DataType.TEXT: 'text'>, score=None)}, {'chunk_id': '37', 'score': 0.7165161371231079, 'chunk': 
Chunk(chunk_id=37, content='![](_page_7_Figure_0.jpeg) <span id="page-7-0"></span>**FIGURE 2** Global input 
importance via mean SHAP values. Ranking of the model input for the most influential to the less influential for 
the model. The y-axis indicates the average change in the predicted probability of edema by grade, on average 
across the entire test set. SHAP, Shapley Additive exPlanations. <span id="page-7-1"></span>**FIGURE 3** SHAP 
values – contribution of the inputs toward the predicted probabilities of edemas of grade 2+. List of the eight 
most influential inputs with respect to the predicted probabilities of edemas of grades 2+. Each point on the plot 
is a SHAP value for a covariate at a specific patient visit. The position on the y-axis indicates the covariate 
importance and on the x-axis the impact on the predicted probability. Color represents the value of the covariate. 
SHAP, Shapley Additive exPlanations. with an increase in the predicted probability of edemas of grades 2+.', 
metadata={'source_text': 'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 37}, 
data_type=<DataType.TEXT: 'text'>, score=None)}, {'chunk_id': '42', 'score': 0.7311644554138184, 'chunk': 
Chunk(chunk_id=42, content='The presence of such input is ensuring high model performances when predicting 
persistence of a given edema grade. The sensitivity analysis to the inclusion of this input revealed a decline in 
the mean cross-validation F1 score of ~0.350 when it was removed as a candidate predictor. However, the use of 
Isotonic Regressions ensures the estimations of correctly calibrated probabilities. For instance, while low 
predicted probabilities, for example, for edemas of grade 2+, might lead to a classification error, they can still 
provide valuable details on existing risk of occurrence of an adverse event (Figure [S5\\)](#page-11-0). Moreover, 
sensitivity analysis together with the SHAP importance highlighted interesting patterns with respect to the 
exposure-related features. Referencing Figure [3](#page-7-1), lower values of the dose [ *t* − 14 days, *t* ] are 
associated with a decreased probability of edema of grades 2+ at subsequent visits.', metadata={'source_text': 
'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 42}, data_type=<DataType.TEXT: 
'text'>, score=None)}, {'chunk_id': '32', 'score': 0.7571419477462769, 'chunk': Chunk(chunk_id=32, 
content='Consistently with the above sensitivity analysis, past current edema grade was found to be the most 
influential input, particularly if a same grade persisted to the following safety visit. The exposure-derived 
features were also informative for the model probability predictions. Albumin was found as the

[Step 1: Duration 3.67 seconds| Input tokens: 2,056 | Output tokens: 138]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_influential_factors = [                                                                                     
      "Current edema grade, especially persistence of Grade 2+",                                                   
      "Low albumin levels",                                                                                        
      "Older age (greater than 70 years)",                                                                         
      "Cumulated dose over the previous 14 days normalized (with lower doses indicating dose adjustments)"         
  ]                                                                                                                
                                                                                                                   
  final_answer(most_influential_factors)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['Current edema grade, especially persistence of Grade 2+', 'Low albumin levels', 'Older age 
(greater than 70 years)', 'Cumulated dose over the previous 14 days normalized (with lower doses indicating dose 
adjustments)']

[Step 2: Duration 4.33 seconds| Input tokens: 5,724 | Output tokens: 346]

In [88]:
output = rag_agent.run("What is the highest court of the USA?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the highest court of the USA?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The highest court of the USA is the Supreme Court of the United States.")                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The highest court of the USA is the Supreme Court of the United States.

[Step 1: Duration 2.41 seconds| Input tokens: 2,042 | Output tokens: 73]

## Exercise 2: Make it a multi-agent system

Now that we built a RAG agent, we will improve it by adding a web search tool to it. This way, if the agent can't find the answer in the knowledge base, it will search the web for it.

However, we will transform our agent into a multi-agent system. This way, we will have one agent responsible for answering questions using the knowledge base and another agent responsible for searching the web.

1. Define a tool that searches the web
2. Define a new agent that uses the web search tool
3. Create a multi-agent system that uses both agents


In [90]:
multi_rag_agent = ...

In [96]:


#
from smolagents import DuckDuckGoSearchTool

web_search_agent = CodeAgent(
    name="web_wikipedia_agent",
    description="An agent that can visit webpage. Try to mostly use it to visit wikipedia when the question is not about medical knowledge. Call it as a function using the **task** argument.",
    tools=[visit_webpage_tool],
    add_base_tools=False,
    model=model,
    verbosity_level=1,
)

rag_agent = CodeAgent(
    name="medical_literature_agent",
    description="Retrieve information from medical litterature. Call it as a function using the **task** argument.",
    tools=[retriever_tool],
    model=model,
)

multi_rag_agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[rag_agent, web_search_agent],
    verbosity_level=1,
)



In [97]:
# Call the agent
output = multi_rag_agent.run(
    "According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  response = medical_literature_agent(task="Provide information from medical literature about SHAP analysis        
  results identifying the most influential factors in predicting higher-grade edema (Grade 2+). Include the top    
  factors reported and any relevant context or insights from the analysis.")                                       
  print(response)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────── New run - medical_literature_agent ───────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'medical_literature_agent'.                                                        │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Provide information from medical literature about SHAP analysis results identifying the most influential        │
│ factors in predicting higher-grade edema (Grade 2+). Include the top factors reported and any relevant context  │
│ or insights from the analysis.                                                                                  │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 21.34 seconds| Input tokens: 3,839 | Output tokens: 621]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_influential_factors = (                                                                                     
      "The most influential factors in predicting higher-grade edema (Grade 2+) "                                  
      "according to SHAP analysis are:\n"                                                                          
      "- Current edema grade (especially persistence of the same grade)\n"                                         
      "- Albumin levels (lower levels increase risk)\n"                                                            
      "- Age, particularly over 70 years\n"                                                                        
      "- Exposure-related features including short-term cumulative dose (higher dose linked to higher risk) "      
      "and long-term cumulative dose (higher dose linked to reduced risk)\n"                                       
  )                                                                                                                
                                                                                                                   
  final_answer(most_influential_factors)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The most influential factors in predicting higher-grade edema (Grade 2+) according to SHAP 
analysis are:
- Current edema grade (especially persistence of the same grade)
- Albumin levels (lower levels increase risk)
- Age, particularly over 70 years
- Exposure-related features including short-term cumulative dose (higher dose linked to higher risk) and long-term 
cumulative dose (higher dose linked to reduced risk)

[Step 2: Duration 4.79 seconds| Input tokens: 6,658 | Output tokens: 827]

In [98]:
output = multi_rag_agent.run("What is the revenue of Nvidia?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the revenue of Nvidia?                                                                                  │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_revenue = web_wikipedia_agent(task="What is the current revenue of Nvidia?")                              
  print(nvidia_revenue)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────────── New run - web_wikipedia_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_wikipedia_agent'.                                                             │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ What is the current revenue of Nvidia?                                                                          │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 22.96 seconds| Input tokens: 14,798 | Output tokens: 427]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Nvidia's current revenue is approximately 130.57 billion U.S. dollars for the fiscal year 2025.")  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Nvidia's current revenue is approximately 130.57 billion U.S. dollars for the fiscal year 2025.

[Step 2: Duration 1.78 seconds| Input tokens: 17,287 | Output tokens: 513]